In [ ]:
from sibyl.core import Sibyl
from mongoengine import disconnect

In [ ]:
database = "housing"

In [ ]:
disconnect()
config = {
    "mongodb": {
        "db": database,
        "host": "localhost",
        "port": 27017,
        "username": None,
        "password": None,
    },
    "log_filename": "test.csv",
    "feature_distribution_location": None,
    "flask": {},
}
explorer = Sibyl(config, docker=False)
app = explorer._init_flask_app("test")
client = app.test_client()

In [ ]:
# TEST ENTITY APIs
eid = None

response = client.get("/api/v1/entities/")
assert response.status_code == 200
assert "entities" in response.json
print("Number of entities:", len(response.json["entities"]))

if eid is None:
    eid = response.json["entities"][0]["eid"]
eids = [response.json["entities"][i]["eid"] for i in range(0, 3)]

response = client.get("/api/v1/entities/" + eid + "/")
assert response.status_code == 200
assert response.json["eid"] == eid
assert "features" in response.json
assert "property" in response.json
sample_entity = response.json

In [ ]:
# TEST FEATURE APIs

response = client.get("/api/v1/features/")
assert response.status_code == 200
assert "features" in response.json
print("Number of features:", len(response.json["features"]))
sample_feature = response.json["features"][0]

response = client.get("/api/v1/features/" + sample_feature["name"] + "/")
assert response.status_code == 200
assert "name" in response.json
print("Sample feature:", response.json)

response = client.get("/api/v1/categories/")
assert response.status_code == 200
assert "categories" in response.json
print("Sample category", response.json["categories"][0])

In [ ]:
# TEST MODEL APIs
response = client.get("/api/v1/models/")
assert response.status_code == 200
print("Number of models:", len(response.json["models"]))
sample_model = response.json["models"][0]
model_id = sample_model["model_id"]

response = client.get("/api/v1/models/" + model_id + "/")
assert response.status_code == 200
print("Sample model:", response.json)

response = client.get("/api/v1/importance/?model_id=" + model_id)
assert response.status_code == 200
assert "importances" in response.json
print("Sample importance", next(iter(response.json["importances"].items())))

response = client.get("/api/v1/prediction/?model_id=" + model_id + "&eid=" + eid)
assert response.status_code == 200
print("Sample prediction:", response.json)
prediction = response.json["output"]

In [ ]:
# TEST CONTEXTS

response = client.get("/api/v1/contexts/")
assert response.status_code == 200
context_id = response.json["contexts"][0]["id"]

response = client.get("/api/v1/context/" + context_id + "/")
assert response.status_code == 200
print("Sample context:", response.json)

In [ ]:
# TEST COMPUTING
import pandas as pd

response = client.post("/api/v1/contributions/", json={"eid": eid, "model_id": model_id})
assert response.status_code == 200
assert "result" in response.json
print("Sample contribution", next(iter(response.json["result"].items())))

response = client.post("/api/v1/multi_contributions/", json={"eids": eids, "model_id": model_id})
assert response.status_code == 200
assert "contributions" in response.json
for eid in response.json["contributions"]:
    try:
        pd.DataFrame.from_dict(response.json["contributions"][eid], orient="index")
    except Exception as e:
        print("Error with multi-contributions - wrong format")
        raise e
print("Multi-contributions validated")

row_ids = list(sample_entity["features"].keys())
features = list(sample_entity["features"][row_ids[0]].keys())

changes = {
    features[0]: sample_entity["features"][row_ids[0]][features[0]],
    features[1]: sample_entity["features"][row_ids[0]][features[1]],
}
response = client.post(
    "/api/v1/modified_prediction/", json={"eid": eid, "model_id": model_id, "changes": changes}
)
assert response.status_code == 200
assert "prediction" in response.json
print("Sample modified prediction:", response.json)

response = client.post(
    "/api/v1/single_change_predictions/",
    json={"eid": eid, "model_id": model_id, "changes": changes},
)
assert response.status_code == 200
assert "predictions" in response.json
print("Sample predictions:", response.json["predictions"])

In [ ]:
# TEST COMPUTING - PART 2
response = client.post(
    "/api/v1/modified_contribution/",
    json={"eid": eid, "model_id": model_id, "changes": changes},
)
assert response.status_code == 200
assert "contribution" in response.json
try:
    df = pd.DataFrame.from_dict(response.json["contribution"], orient="index")
    print("Sample contribution object:\n", df[df.columns[0]])
except Exception as e:
    print("Error with modified-contributions - wrong format")
    raise e

response = client.post("/api/v1/similar_entities/", json={"eids": eids, "model_id": model_id})
assert response.status_code == 200
assert "similar_entities" in response.json
print_flag = True
for eid in response.json["similar_entities"]:
    assert "X" in response.json["similar_entities"][eid]
    try:
        X = pd.DataFrame.from_dict(
            response.json["similar_entities"][eid]["X"], orient="index"
        ).head()
        if print_flag:
            print("Sample similar entity:\n", X)
    except Exception as e:
        print("Error with similar entities - wrong X format")
        raise e
    assert "y" in response.json["similar_entities"][eid]
    try:
        y = pd.Series(response.json["similar_entities"][eid]["y"]).head()
        if print_flag:
            print("y:\n", y)
    except Exception as e:
        print("Error with similar entities - wrong y format")
        raise e
    try:
        input = pd.Series(response.json["similar_entities"][eid]["Input"]).head()
        if print_flag:
            print("Input row:\n", input)
            print_flag = False
    except Exception as e:
        print("Error with similar entities - wrong input format")
        raise e